<div style="font-family: 'JetBrains Mono', monospace; font-size: 14px; color: #e2dbdbff; line-height: 1.6;">

## 📊 Exploratory Data Analysis (EDA)

### ✅ Principales insights:

- **25.29%** de los comercios tienen presencia web.
- **Tandil** y **Olavarria** lideran en digitalización.
- **Instagram 19.2%** y **Facebook 18.11** concentran la presencia en RRSS
- Los comercios con sitio web tienen **mayor cantidad de reviews** en todas las ciudades.
- El puntaje promedio se mantiene **alto y constante (4.0 - 4.4)**.
- Categorías más comunes:
  - 
  - 
  - 
  - 

---

</div>

In [26]:
import pandas as pd
import os
import numpy as np
from glob import glob
import glob
import matplotlib.pyplot as plt
import seaborn as sns

# Configura pandas para mostrar todas las columnas del DataFrame al mostrarlo, sin recorte visual.
pd.set_option('display.max_columns', None) 
# Configuración global para TODOS los DataFrames
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 200)  # Máximo caracteres por columna

In [17]:
df = pd.read_csv('../Python_es/CleanData/Regional.csv')

In [18]:
df.shape

(5859, 16)

In [19]:
df.size

93744

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5859 entries, 0 to 5858
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         5859 non-null   object 
 1   categoryName  5859 non-null   object 
 2   website       5859 non-null   object 
 3   street        5859 non-null   object 
 4   phone         5859 non-null   object 
 5   city          5859 non-null   object 
 6   state         5859 non-null   object 
 7   location/lat  5859 non-null   float64
 8   location/lng  5859 non-null   float64
 9   totalScore    5859 non-null   float64
 10  rank          5859 non-null   int64  
 11  imageUrl      5859 non-null   object 
 12  imagesCount   5859 non-null   float64
 13  reviewsCount  5859 non-null   int64  
 14  scrapedAt     5859 non-null   object 
 15  searchString  5859 non-null   object 
dtypes: float64(4), int64(2), object(10)
memory usage: 732.5+ KB


In [21]:
df.dtypes

title            object
categoryName     object
website          object
street           object
phone            object
city             object
state            object
location/lat    float64
location/lng    float64
totalScore      float64
rank              int64
imageUrl         object
imagesCount     float64
reviewsCount      int64
scrapedAt        object
searchString     object
dtype: object

In [22]:
# Convertir fecha de scraping manteniendo formato datetime
# Esto permite análisis temporales futuros (ej: comparar con scraping en 6 meses)
df['scrapedAt'] = pd.to_datetime(
    df['scrapedAt'], 
    errors='coerce',  # Maneja fechas inválidas sin romper el código
    utc=True          # Estandariza zona horaria para comparaciones
).dt.floor('s').dt.tz_localize(None)  # Elimina timezone para visualización limpia
# Resultado: 2025-06-15 22:56:55
df['scrapedAt']

0      2025-06-15 22:56:55
1      2025-06-15 22:56:54
2      2025-06-15 22:56:57
3      2025-06-15 22:56:57
4      2025-06-15 22:56:58
               ...        
5854   2025-07-30 15:45:05
5855   2025-07-30 15:45:19
5856   2025-07-30 15:45:19
5857   2025-07-30 15:45:06
5858   2025-07-30 15:45:04
Name: scrapedAt, Length: 5859, dtype: datetime64[ns]

In [23]:
df.head(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
0,Dti - Azul,Soporte Y Servicios Informáticos,Incomplete,Av. Juan Domingo Perón 525,+54 9 2281 42-7066,Azul,Provincia de Buenos Aires,-36.782006,-59.865497,0.0,22,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=zL2SLXDLlIyLOOkG7pxFlg&cb_client=search.gws-prod.gps&w=408&h=240&yaw=231.62384&pitch=0&thumbfov=100,1.0,0,2025-06-15 22:56:55,Tecnologia
1,Tecno Computacion,Tienda De Móviles,http://www.tecnocomputacionazul.com/,Lamadrid 33,+54 9 2281 65-7938,Azul,Provincia de Buenos Aires,-36.764914,-59.862404,4.6,3,https://lh3.googleusercontent.com/p/AF1QipP9GgrGu9iOG5xNOUsuEUj6fdq4j3xvaLptITLH=w408-h408-k-no,12.0,7,2025-06-15 22:56:54,Tecnologia
2,Ingeniero Marcelo Edgardo Cornec,Ingeniero,Incomplete,Av. Juan Domingo Perón 622,+54 9 2281 42-6984,Azul,Provincia de Buenos Aires,-36.783278,-59.863594,5.0,20,https://lh3.googleusercontent.com/p/AF1QipN3mw2facF8IFrTvwJ_gdFhsNWQrFQi4pazN5RN=w426-h240-k-no,13.0,3,2025-06-15 22:56:57,Comercio
3,La Esquina,Kiosco,Incomplete,"7300, Gral. Paz 1500",+54 9 2281 36-0736,Azul,Provincia de Buenos Aires,-36.798090,-59.853525,3.7,19,https://lh3.googleusercontent.com/p/AF1QipP1o8LwpKf0khxVBfihAWLOI4K4X00X3SCtGAg=w408-h544-k-no,1.0,3,2025-06-15 22:56:57,Comercio
4,Vidrieria 25 De Mayo De Jose Canalicchio,Comercio,Incomplete,Guaminí 610,+54 9 2281 53-7036,Azul,Provincia de Buenos Aires,-36.785552,-59.867145,4.2,40,https://lh3.googleusercontent.com/p/AF1QipM1o-ulafI-mAMROrwDJeAv15XDqrWz-t4wzI86=w426-h240-k-no,5.0,9,2025-06-15 22:56:58,Comercio
5,El Almacén,Tienda De Ropa,Incomplete,Av. Mitre 901,+54 9 2281 43-6001,Azul,Provincia de Buenos Aires,-36.782064,-59.853528,4.1,37,https://lh3.googleusercontent.com/gps-cs-s/AC9h4nqJ1zIBol2hSMqViHOfQ4nImOXxIbkIgBdKGU_mDledq073-DS2ESZ6a9J8zJbFBVcYlP_NC7ylDTrU9a76gRU5qeBFyq79Kl8GzNNI_mskXAlESWWmTl8R-huNvIgRMaK2F812zQ=w408-h306-...,2.0,15,2025-06-15 22:56:58,Comercio
6,Tus Sorrentinos,Comercio,Incomplete,España 418,+54 9 2281 56-4122,Azul,Provincia de Buenos Aires,-36.779354,-59.853713,0.0,36,https://lh3.googleusercontent.com/p/AF1QipPp_by7_7ex8dl9zbGenhIxYdoDgTEiYRtP-AWu=w408-h906-k-no,22.0,0,2025-06-15 22:56:58,Comercio
7,Venden Y Permutas,Tienda General,Incomplete,Gral. Rivas 787,Incomplete,Azul,Provincia de Buenos Aires,-36.788767,-59.865031,0.0,34,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=S1kTFpHnfYoL20C_61-NkA&cb_client=search.gws-prod.gps&w=408&h=240&yaw=198.58519&pitch=0&thumbfov=100,1.0,0,2025-06-15 22:56:58,Comercio
8,Minimercado La Esquina De Alejandro Y Silvina,Comercio,Incomplete,España 1699,Incomplete,Azul,Provincia de Buenos Aires,-36.796488,-59.873790,4.3,29,https://lh3.googleusercontent.com/gps-cs-s/AC9h4nqf9tsq_AytWExGgJrpTk-L5SI1FmirJVvwRjC9ojTeSPOq6No9fkPU0XwnPnweLQIPGS1cYMnUkXjZSSbWHXJM-5agZX8IO8O6YdfxK7B9pWqmEzDTfcSIynJmCjEkKcpC9s8Hz9Ucyb5x=w408...,20.0,22,2025-06-15 22:56:58,Comercio
9,Kiosco Pasaje Del Parque,Comercio,Incomplete,Pje. 25 de Abril,+54 9 2281 52-6194,Azul,Provincia de Buenos Aires,-36.781831,-59.875032,5.0,23,https://lh3.googleusercontent.com/gps-cs-s/AC9h4nrTY3uDxVEBPRv2VqYKJaV-GoWN44PK80cq9eCCZxf0BdsC3IIWztwGi8v8yX136XorfRDmD_52j0ECvAdutl4_BLUUwCM-JpnySErvCbimv3ztqqXm4Km6u_xq8IvG-_Yx57VYEWNB1ZdW=w408...,6.0,1,2025-06-15 22:56:58,Comercio


In [24]:
df.tail(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
5849,Mundo Peugeot - Repuestos Y Accesorios,Tienda De Repuestos Para Automóviles,Incomplete,Gral. Rodríguez 1655,+54 9 249 402-4966,Tandil,Provincia de Buenos Aires,-37.313746,-59.143988,4.5,75,https://lh3.googleusercontent.com/p/AF1QipPA8V3z3nu3v95en7me-WKWla3MO1HsHUIh8Hi2=w408-h408-k-no,11.0,86,2025-07-30 15:45:17,Repuestos
5850,Riders Repuestos,Tienda De Repuestos Para Automóviles,Incomplete,Gral. Rodríguez 1495,Incomplete,Tandil,Provincia de Buenos Aires,-37.316190,-59.142774,0.0,70,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=IkLlCsFgI0TI6p8PROI1wg&cb_client=search.gws-prod.gps&w=408&h=240&yaw=272.69937&pitch=0&thumbfov=100,1.0,0,2025-07-30 15:45:17,Repuestos
5851,Santamarina Repuestos,Tienda De Repuestos Para Automóviles,Incomplete,Av. Santamarina 36,Incomplete,Tandil,Provincia de Buenos Aires,-37.331030,-59.126801,0.0,69,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=CouOChX2xFSfhWB8jtw4HQ&cb_client=search.gws-prod.gps&w=408&h=240&yaw=230.9067&pitch=0&thumbfov=100,1.0,0,2025-07-30 15:45:17,Repuestos
5852,Repuestos Buzon,Tienda De Repuestos Para Automóviles,Incomplete,Av. Balbín 1073,+54 9 249 448-7106,Tandil,Provincia de Buenos Aires,-37.313849,-59.122081,4.9,66,https://lh3.googleusercontent.com/gps-proxy/ALd4DhFIShJPEp_8LA2VCdhd1f49JhdHWDClE0HcmCO5XCBJjiRffUw6uLxBFp5LxEaIxOz29zVPlxgFalwDsyJZeKWujyCckcDNuD_6Sme7Fivil72-fFaRRCdH2G1YoP6vmRW_cnLM1aks7fiMwPE5...,2.0,49,2025-07-30 15:45:17,Repuestos
5853,Kain Repuestos Gas Y Agua,Tienda De Artículos Relacionados Con El Gas,Incomplete,Leandro Alem 1217,+54 9 249 425-1165,Tandil,Provincia de Buenos Aires,-37.317811,-59.138120,4.8,68,https://lh3.googleusercontent.com/p/AF1QipNt9jw67BsSDAuQ9PtiX6SdIKTQA6Aobm3ACpa3=w408-h542-k-no,65.0,5,2025-07-30 15:45:17,Repuestos
5854,Arkansas Muebles,Tienda De Muebles,http://arkansastandil.mitiendanube.com/,Av. Marconi 1498,+54 9 249 464-3311,Tandil,Provincia de Buenos Aires,-37.317572,-59.125688,5.0,79,https://lh3.googleusercontent.com/gps-proxy/ALd4DhE1EbgcrMM7_msEKZ5Fs4WQdrdgiI950eTOUaZ6wp8u9HNkIFZjqTJMMOmUxmPC1rwq7of0HLqBCqCfpiVDTkNL_xaBen-u1bl_BhCzySL2f-oEY27OdKLzuFXpFaqzqc_KtUAjn-EpNScOF1n8...,3.0,1,2025-07-30 15:45:05,Electrodomesticos
5855,Pilchas Mb,Tienda De Ropa De Hombre,Incomplete,San Martín 915,+54 9 223 555-9393,Tandil,Provincia de Buenos Aires,-37.324699,-59.132030,5.0,78,https://lh3.googleusercontent.com/gps-cs-s/AC9h4non8uQP_sqcqowOu_UsOqxgV0SYhiPMiy8CLWiITFzTjfeqRyEvV0bwxJA46ejqnVe_K4i2fu3HYvqiip0jrLJTGeXMhzQOqzUACcv913IRE1rFxu3Qp7sXosd1H3xoE5WM0n3eZQfvtpjf=w408...,6.0,9,2025-07-30 15:45:19,Ropa
5856,Rodriggi,Tienda De Ropa De Hombre,Incomplete,9 de Julio 445,+54 9 249 457-5285,Tandil,Provincia de Buenos Aires,-37.327796,-59.133822,4.9,80,https://lh3.googleusercontent.com/p/AF1QipPNSwjqO-BcGWZ_QFWc4MR_7a0o8wZy3S1BjG54=w408-h272-k-no,6.0,23,2025-07-30 15:45:19,Ropa
5857,Lg Service,Servicio De Reparación De Electrodomésticos,Incomplete,Saavedra 778,+54 9 249 438-8237,Tandil,Provincia de Buenos Aires,-37.318255,-59.121669,4.7,84,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=w8F2RrnwTIVnJD4m2ClR_Q&cb_client=search.gws-prod.gps&w=408&h=240&yaw=255.47885&pitch=0&thumbfov=100,1.0,3,2025-07-30 15:45:06,Electrodomesticos
5858,Friototal,Distribuidor De Acero,http://www.friototal.com.ar/,Av. del Valle 1258,+54 9 249 469-8239,Tandil,Provincia de Buenos Aires,-37.309298,-59.134556,3.7,60,https://lh3.googleusercontent.com/gps-cs-s/AC9h4nrY6dCcXjno432CvCOQjpVB1uui0qruYbu1hwIZ9zCq_uIbqnlbWlMGFcmKVIPH_OtBeioELjwT2aCv0NjcR8GnMqs9hQt__70e2fvlxQY2zTEhsB8tGw2PXg77qTN7BxzMP-qDTA=w408-h544-...,17.0,3,2025-07-30 15:45:04,Electrodomesticos


In [25]:
df.sample(10)

,title,categoryName,website,street,phone,city,state,location/lat,location/lng,totalScore,rank,imageUrl,imagesCount,reviewsCount,scrapedAt,searchString
4960,Tienda De Aromas,Tienda De Aromaterapia,https://linkr.bio/tiendadearomas.tandil&opi=79508299&sa=U&ved=0ahUKEwjv_8Xi-emNAxWiSDABHb8ECUQQ61gIvgEoEQ&usg=AOvVaw2SY5yS9sUwjE8kOk5ZfoN5,San Martín 645,+54 9 249 420-9304,Tandil,Provincia de Buenos Aires,-37.326256,-59.136184,4.8,25,https://lh3.googleusercontent.com/p/AF1QipN2NVtV_Y8spb8aswk9iSBItiNiYSL4Qgv5mQ7Q=w408-h285-k-no,15.0,19,2025-06-11 18:02:05,Tienda
2612,Producciones Lina,Servicio De Reparación De Ordenadores,Incomplete,"Local 13, Galería Vicente López, Vicente López 2865",+54 9 2284 67-1694,Olavarría,Provincia de Buenos Aires,-36.892650,-60.322859,0.0,89,https://lh3.googleusercontent.com/gps-cs-s/AC9h4noM6MjI2jP-kPCCBnGip4YI-qs9CFFgOl-m4SMbTe9DweawNnGHM2jt7xHV1JUOTNc3RwQoSs55gBEZ4Hz29YYqgnYl45nifCKsDabgo0MFsmuI0KVg4AMPlApFllVncW6fBTaD=w408-h306-k-no,29.0,0,2025-07-30 14:42:45,Hogar
441,Deportivo A Club,Club,Incomplete,Lavalle 573,+54 9 2281 42-3673,Azul,Provincia de Buenos Aires,-36.783398,-59.853947,5.0,14,https://lh3.googleusercontent.com/gps-cs-s/AC9h4noS9zdvlLKW3ce5YAT-gN22nRNDGFX-mYaZS7d7O0gujwgjtRBo6sDfglEXMG59Te5uuQT2Ak5GEcZKtM91Nm3b8adXItyQD9OcZQoOrcNY5u_Hh2sXCZIbfB5Me_ShzULjmcdxsA=w426-h240-...,3.0,3,2025-07-30 14:51:18,Deportivo
879,Despensa Ruta 3,Tienda De Alimentación,Incomplete,"KM 301, RN3",Incomplete,Azul,Provincia de Buenos Aires,-36.804569,-59.855364,5.0,84,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=ThXrzYtTTRhJWvCghBOEFw&cb_client=search.gws-prod.gps&w=408&h=240&yaw=244.43944&pitch=0&thumbfov=100,1.0,1,2025-07-30 14:52:06,Comercio
2982,Club Social Y Deportivo Unión Del Sud,Club,http://www.clubargentinosdelsud.blogspot.com/,Av. del Valle 3853,Incomplete,Olavarría,Provincia de Buenos Aires,-36.905062,-60.319443,2.8,13,https://lh3.googleusercontent.com/gps-cs-s/AC9h4nrZW48VG-T3r1S-bIo5JgPP6WfDKXXAmvmQzXpjg4X7aRM2k8VMMjvMjIZtbH-aIIYX8DuAPloaTGA2Uwyb8FRFZvg5Y5QY7LPh2o4dsTr6cxnDL6MVeEQ0csDwrm6O8bM7rUO0cQ=w426-h240-...,7.0,4,2025-07-30 14:43:14,Deportivo
3092,Bambini,Tienda De Ropa Infantil,Incomplete,Necochea 3121,+54 9 2284 58-9010,Olavarría,Provincia de Buenos Aires,-36.893934,-60.317061,5.0,81,https://lh3.googleusercontent.com/p/AF1QipOYkux6Jx5Ou4rKT8HlFuFWVMEvsVHBb79ZdyMV=w408-h256-k-no,3.0,14,2025-07-30 14:43:33,Ropa
105,Direccion De Regularización Dominial,Departamento De Obras Públicas,Incomplete,Av. 25 de Mayo 647,Incomplete,Azul,Provincia de Buenos Aires,-36.780202,-59.860594,0.0,58,https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=lRzhTQ0LMqcg3Q0Jt9NjKw&cb_client=search.gws-prod.gps&w=408&h=240&yaw=130.6616&pitch=0&thumbfov=100,1.0,0,2025-06-15 22:57:24,Venta
5431,Casa Cina-Cinas,Urbanización,Incomplete,Los Fresnos 657,Incomplete,Tandil,Provincia de Buenos Aires,-37.324161,-59.091507,0.0,11,Incomplete,0.0,0,2025-07-30 15:42:56,Inmobiliario
1133,Aquellarre,Joyería,Incomplete,Constitución 991,Incomplete,Azul,Provincia de Buenos Aires,-36.792122,-59.856401,0.0,32,Incomplete,0.0,0,2025-07-30 14:53:43,Tienda
1684,Egno Shop,Tienda De Alimentación,http://www.facebook.com/egnoshop,Av. Colón,+54 9 2284 60-9993,Olavarría,Provincia de Buenos Aires,-36.887514,-60.320647,5.0,7,https://lh3.googleusercontent.com/gps-proxy/ALd4DhGrTk2BYcozS4BX2nLAzRtPtV54rN-EQ2cFRqSoxF_fuiP-eqfnPvcR5VTk8NzGX_iq1iy3NbaxhmqKNeDWMS1jd5Gob1-0UUsK3WH1mdZj7KMOvY5P_umUnyVGx34YUMkZrrfEhsV7Mo3ZJ0Kw...,4.0,1,2025-05-29 00:56:15,Tienda
